# **Curating `fetal-lung` dataset:**

### Organoid dataset

# <ins>STEPS:</ins>

## 1. Load the dataset 

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy

In [113]:
adata2 = sc.read_h5ad('/lustre/scratch117/cellgen/cellgeni/cxgportal_sets/fetal-lung/Annotated_OE_Organoid_Filtered.h5ad')

In [117]:
len(adata2.var['highly_variable_n-in vitro'])

25741

In [8]:
adata.var = adata.var[['n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 
                       'gene_ids-WSSS_F_LNG10282020-in vivo', 'dispersions-in vitro', 'dispersions_norm-in vitro', 'n_cells-in vivo', 'highly_variable_n-in vitro', 'highly_variable-in vivo', 'Deep_n-in vivo']]

In [9]:
adata.var

,n_cells-in vitro,highly_variable_n-in vitro,highly_variable-in vitro,Deep_n-in vitro,gene_ids-WSSS_F_LNG10282020-in vivo,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable_n-in vitro,highly_variable-in vivo,Deep_n-in vivo
A1BG,250.0,13.0,False,0.0,ENSG00000121410,0.978903,0.277938,3826.0,13.0,False,1.0
A1BG-AS1,39.0,2.0,False,1.0,ENSG00000268895,0.642069,-0.270407,554.0,2.0,False,0.0
A1CF,35.0,0.0,False,0.0,ENSG00000148584,1.004072,0.318912,5855.0,0.0,True,0.0
A2M,359.0,28.0,True,21.0,ENSG00000175899,2.273938,2.386176,244.0,28.0,False,0.0
A2M-AS1,96.0,7.0,False,3.0,ENSG00000245105,0.731219,-0.125276,134.0,7.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,2290.0,14.0,False,0.0,ENSG00000162378,0.943422,0.220178,11061.0,14.0,True,0.0
ZYX,2975.0,9.0,False,9.0,ENSG00000159840,0.915690,0.175031,42173.0,9.0,False,0.0
ZZEF1,1679.0,21.0,True,0.0,ENSG00000074755,1.253851,0.725536,15245.0,21.0,True,0.0
deltaNp63alpha_trans,NaN,NaN,nan,NaN,NaN,NaN,NaN,10366.0,NaN,False,1.0


In [10]:
def CalculateRaw(adata,scaling_factor=10000):
    #update by Polanski in Feb 2022
    #The object must contain a log-transformed matrix
    #This function returns an integer-count object
    #The normalization constant is assumed to be 10000
    #return anndata.AnnData(X=sparse.csr_matrix(np.rint(np.array(np.expm1(adata.X).todense().transpose())*(adata.obs['n_counts'].values).transpose() / scaling_factor).transpose()),\
    #              obs=adata.obs,var=adata.var,obsm=adata.obsm,varm=adata.varm)
    X = np.expm1(adata.X)
    scaling_vector = adata.obs['n_counts'].values / scaling_factor
    #.indptr[i]:.indptr[i+1] provides the .data coordinates where the i'th row of the data resides in CSR
    #which happens to be a cell, which happens to be what we have a unique entry in scaling_vector for
    for i in np.arange(X.shape[0]):
        X.data[X.indptr[i]:X.indptr[i+1]] = X.data[X.indptr[i]:X.indptr[i+1]] * scaling_vector[i]
    return sc.AnnData(X=np.rint(X),obs=adata.obs,var=adata.var,obsm=adata.obsm,varm=adata.varm)

In [11]:
raw = CalculateRaw(adata)

In [12]:
del raw.var

In [13]:
geno = pd.read_csv('~/genes_homo_sapiens.csv', header = None, names = ['ENSEMBLid', 'gena','no'])

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
geno

,ENSEMBLid,gena,no
0,ENSG00000223972,DDX11L1,5
1,ENST00000456328,DDX11L1-202,2
2,ENST00000450305,DDX11L1-201,2
3,ENSG00000227232,WASH7P,5
4,ENST00000488147,WASH7P-201,1
...,...,...,...
297782,ENST00000621424,ENSG110105435.4-001,4
297783,ENSG00000278625,U6,1
297784,ENST00000616830,U6.88-201,1
297785,ENSG00000277374,U1,1


In [15]:
ens_ids = list(adata.var['gene_ids-WSSS_F_LNG10282020-in vivo'])

In [16]:
ens_ids

['ENSG00000121410',
 'ENSG00000268895',
 'ENSG00000148584',
 'ENSG00000175899',
 'ENSG00000245105',
 'ENSG00000166535',
 nan,
 'ENSG00000184389',
 'ENSG00000128274',
 'ENSG00000118017',
 'ENSG00000094914',
 'ENSG00000081760',
 'ENSG00000114771',
 'ENSG00000109576',
 'ENSG00000158122',
 'ENSG00000103591',
 'ENSG00000115977',
 'ENSG00000087884',
 'ENSG00000127837',
 'ENSG00000129673',
 'ENSG00000131043',
 'ENSG00000205002',
 'ENSG00000090861',
 'ENSG00000124608',
 'ENSG00000266967',
 'ENSG00000157426',
 'ENSG00000149313',
 'ENSG00000008311',
 'ENSG00000215458',
 'ENSG00000275700',
 'ENSG00000181409',
 'ENSG00000254180',
 'ENSG00000281376',
 'ENSG00000183044',
 'ENSG00000165029',
 'ENSG00000154263',
 'ENSG00000144452',
 'ENSG00000179869',
 'ENSG00000107331',
 'ENSG00000167972',
 'ENSG00000198691',
 'ENSG00000154265',
 'ENSG00000154262',
 'ENSG00000064687',
 'ENSG00000141338',
 'ENSG00000154258',
 'ENSG00000231749',
 'ENSG00000085563',
 'ENSG00000135776',
 'ENSG00000073734',
 'ENSG00000005

In [17]:
adata.var

,n_cells-in vitro,highly_variable_n-in vitro,highly_variable-in vitro,Deep_n-in vitro,gene_ids-WSSS_F_LNG10282020-in vivo,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable_n-in vitro,highly_variable-in vivo,Deep_n-in vivo
A1BG,250.0,13.0,False,0.0,ENSG00000121410,0.978903,0.277938,3826.0,13.0,False,1.0
A1BG-AS1,39.0,2.0,False,1.0,ENSG00000268895,0.642069,-0.270407,554.0,2.0,False,0.0
A1CF,35.0,0.0,False,0.0,ENSG00000148584,1.004072,0.318912,5855.0,0.0,True,0.0
A2M,359.0,28.0,True,21.0,ENSG00000175899,2.273938,2.386176,244.0,28.0,False,0.0
A2M-AS1,96.0,7.0,False,3.0,ENSG00000245105,0.731219,-0.125276,134.0,7.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,2290.0,14.0,False,0.0,ENSG00000162378,0.943422,0.220178,11061.0,14.0,True,0.0
ZYX,2975.0,9.0,False,9.0,ENSG00000159840,0.915690,0.175031,42173.0,9.0,False,0.0
ZZEF1,1679.0,21.0,True,0.0,ENSG00000074755,1.253851,0.725536,15245.0,21.0,True,0.0
deltaNp63alpha_trans,NaN,NaN,nan,NaN,NaN,NaN,NaN,10366.0,NaN,False,1.0


In [18]:
adata.var

,n_cells-in vitro,highly_variable_n-in vitro,highly_variable-in vitro,Deep_n-in vitro,gene_ids-WSSS_F_LNG10282020-in vivo,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable_n-in vitro,highly_variable-in vivo,Deep_n-in vivo
A1BG,250.0,13.0,False,0.0,ENSG00000121410,0.978903,0.277938,3826.0,13.0,False,1.0
A1BG-AS1,39.0,2.0,False,1.0,ENSG00000268895,0.642069,-0.270407,554.0,2.0,False,0.0
A1CF,35.0,0.0,False,0.0,ENSG00000148584,1.004072,0.318912,5855.0,0.0,True,0.0
A2M,359.0,28.0,True,21.0,ENSG00000175899,2.273938,2.386176,244.0,28.0,False,0.0
A2M-AS1,96.0,7.0,False,3.0,ENSG00000245105,0.731219,-0.125276,134.0,7.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,2290.0,14.0,False,0.0,ENSG00000162378,0.943422,0.220178,11061.0,14.0,True,0.0
ZYX,2975.0,9.0,False,9.0,ENSG00000159840,0.915690,0.175031,42173.0,9.0,False,0.0
ZZEF1,1679.0,21.0,True,0.0,ENSG00000074755,1.253851,0.725536,15245.0,21.0,True,0.0
deltaNp63alpha_trans,NaN,NaN,nan,NaN,NaN,NaN,NaN,10366.0,NaN,False,1.0


In [19]:
adata

AnnData object with n_obs × n_vars = 70495 × 25741
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project'
    var: 'n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 'gene_ids-WSSS_F_LNG10282020-in vivo', 'dispersions-in vitro', 'dispersions_norm-in vitro', 'n_cells-in vivo', 'highly_variable_n-in vitro', 'highly_variable-in vivo', 'Deep_n-in vivo'
    uns: 'Souporcell3_assignment_colors', 'Souporcell3_status_colors', 'new_celltype_colors', 'phenotype_colors', 'project_colors'
    obsm: 'X_original_umap', 'X_pca', 'X_umap', 'rep'

In [20]:
adata

AnnData object with n_obs × n_vars = 70495 × 25741
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project'
    var: 'n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 'gene_ids-WSSS_F_LNG10282020-in vivo', 'dispersions-in vitro', 'dispersions_norm-in vitro', 'n_cells-in vivo', 'highly_variable_n-in vitro', 'highly_variable-in vivo', 'Deep_n-in vivo'
    uns: 'Souporcell3_assignment_colors', 'Souporcell3_status_colors', 'new_celltype_colors', 'phenotype_colors', 'project_colors'
    obsm: 'X_original_umap', 'X_pca', 'X_umap', 'rep'

In [21]:
genedict = {key: 1 for key in list(geno.ENSEMBLid)}

In [22]:
genedict

{'ENSG00000223972': 1,
 'ENST00000456328': 1,
 'ENST00000450305': 1,
 'ENSG00000227232': 1,
 'ENST00000488147': 1,
 'ENSG00000278267': 1,
 'ENST00000619216': 1,
 'ENSG00000243485': 1,
 'ENST00000473358': 1,
 'ENST00000469289': 1,
 'ENSG00000284332': 1,
 'ENST00000607096': 1,
 'ENSG00000237613': 1,
 'ENST00000417324': 1,
 'ENST00000461467': 1,
 'ENSG00000268020': 1,
 'ENST00000606857': 1,
 'ENSG00000240361': 1,
 'ENST00000642116': 1,
 'ENST00000492842': 1,
 'ENSG00000186092': 1,
 'ENST00000641515': 1,
 'ENSG00000238009': 1,
 'ENST00000466430': 1,
 'ENST00000477740': 1,
 'ENST00000471248': 1,
 'ENST00000610542': 1,
 'ENST00000453576': 1,
 'ENSG00000239945': 1,
 'ENST00000495576': 1,
 'ENSG00000233750': 1,
 'ENST00000442987': 1,
 'ENSG00000268903': 1,
 'ENST00000494149': 1,
 'ENSG00000269981': 1,
 'ENST00000595919': 1,
 'ENSG00000239906': 1,
 'ENST00000493797': 1,
 'ENSG00000241860': 1,
 'ENST00000484859': 1,
 'ENST00000490997': 1,
 'ENST00000466557': 1,
 'ENST00000662089': 1,
 'ENST00000

In [23]:
(np.nan in genedict)

False

In [24]:
ens_ids2 = [x for x in ens_ids if (x in genedict)]

In [25]:
ens_ids2

['ENSG00000121410',
 'ENSG00000268895',
 'ENSG00000148584',
 'ENSG00000175899',
 'ENSG00000245105',
 'ENSG00000166535',
 'ENSG00000184389',
 'ENSG00000128274',
 'ENSG00000118017',
 'ENSG00000094914',
 'ENSG00000081760',
 'ENSG00000114771',
 'ENSG00000109576',
 'ENSG00000158122',
 'ENSG00000103591',
 'ENSG00000115977',
 'ENSG00000087884',
 'ENSG00000127837',
 'ENSG00000129673',
 'ENSG00000131043',
 'ENSG00000205002',
 'ENSG00000090861',
 'ENSG00000124608',
 'ENSG00000266967',
 'ENSG00000157426',
 'ENSG00000149313',
 'ENSG00000008311',
 'ENSG00000215458',
 'ENSG00000275700',
 'ENSG00000181409',
 'ENSG00000254180',
 'ENSG00000281376',
 'ENSG00000183044',
 'ENSG00000165029',
 'ENSG00000154263',
 'ENSG00000144452',
 'ENSG00000179869',
 'ENSG00000107331',
 'ENSG00000167972',
 'ENSG00000198691',
 'ENSG00000154265',
 'ENSG00000154262',
 'ENSG00000064687',
 'ENSG00000141338',
 'ENSG00000154258',
 'ENSG00000231749',
 'ENSG00000085563',
 'ENSG00000135776',
 'ENSG00000073734',
 'ENSG00000005471',


In [26]:
len(ens_ids2)

24653

In [27]:
adata

AnnData object with n_obs × n_vars = 70495 × 25741
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project'
    var: 'n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 'gene_ids-WSSS_F_LNG10282020-in vivo', 'dispersions-in vitro', 'dispersions_norm-in vitro', 'n_cells-in vivo', 'highly_variable_n-in vitro', 'highly_variable-in vivo', 'Deep_n-in vivo'
    uns: 'Souporcell3_assignment_colors', 'Souporcell3_status_colors', 'new_celltype_colors', 'phenotype_colors', 'project_colors'
    obsm: 'X_original_umap', 'X_pca', 'X_umap', 'rep'

In [28]:
adata.var['gene_symbols'] = adata.var_names
adata.var_names = adata.var['gene_ids-WSSS_F_LNG10282020-in vivo']

In [123]:
adata2.var['gene_symbols'] = adata2.var_names
adata2.var_names = adata2.var['gene_ids-WSSS_F_LNG10282020-in vivo']

In [29]:
raw.var['gene_symbols'] = adata.var_names
raw.var_names = adata.var['gene_ids-WSSS_F_LNG10282020-in vivo']

In [30]:
del adata.var['gene_ids-WSSS_F_LNG10282020-in vivo']

In [31]:
adata = adata[:, ens_ids2].copy()

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [124]:
adata2 = adata2[:, ens_ids2].copy()

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [125]:
adata2

AnnData object with n_obs × n_vars = 70495 × 24653
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project'
    var: 'n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 'gene_ids-in vitro', 'highly_variable5891STDY8062349-in vitro', 'highly_variable5891STDY8062350-in vitro', 'highly_variable5891STDY8062351-in vitro', 'highly_variable5891STDY8062352-in vitro', 'highly_variable5891STDY8062353-in vitro', 'highly_variable5891STDY8062354-in vitro', 'highly_variable5891STDY8062355-in vitro', 'highly_variable5891STDY8062356-in vitro', 'highly_variableWSSS8012016-in vitro', 'highly_variableWSSS8011222-in vitro', 'highly_variableWSSS_F_LNG8713176-

In [32]:
raw = raw[:, ens_ids2].copy()

In [33]:
adata

AnnData object with n_obs × n_vars = 70495 × 24653
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project'
    var: 'n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 'dispersions-in vitro', 'dispersions_norm-in vitro', 'n_cells-in vivo', 'highly_variable_n-in vitro', 'highly_variable-in vivo', 'Deep_n-in vivo', 'gene_symbols'
    uns: 'Souporcell3_assignment_colors', 'Souporcell3_status_colors', 'new_celltype_colors', 'phenotype_colors', 'project_colors'
    obsm: 'X_original_umap', 'X_pca', 'X_umap', 'rep'

In [34]:
raw

AnnData object with n_obs × n_vars = 70495 × 24653
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project'
    var: 'gene_symbols'
    obsm: 'X_original_umap', 'X_pca', 'X_umap', 'rep'

In [35]:
del raw.obs
del raw.obsm

In [36]:
adata.var['feature_biotype'] = ["gene"] * len(adata.var)

In [37]:
raw.var['feature_biotype'] = ["gene"] * len(adata.var)

In [38]:
adata.raw = raw

## 2. Pre-prepare before schema adjustments

In [39]:
adata.var

,n_cells-in vitro,highly_variable_n-in vitro,highly_variable-in vitro,Deep_n-in vitro,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable_n-in vitro,highly_variable-in vivo,Deep_n-in vivo,gene_symbols,feature_biotype
gene_ids-WSSS_F_LNG10282020-in vivo,,,,,,,,,,,,
ENSG00000121410,250.0,13.0,False,0.0,0.978903,0.277938,3826.0,13.0,False,1.0,A1BG,gene
ENSG00000268895,39.0,2.0,False,1.0,0.642069,-0.270407,554.0,2.0,False,0.0,A1BG-AS1,gene
ENSG00000148584,35.0,0.0,False,0.0,1.004072,0.318912,5855.0,0.0,True,0.0,A1CF,gene
ENSG00000175899,359.0,28.0,True,21.0,2.273938,2.386176,244.0,28.0,False,0.0,A2M,gene
ENSG00000245105,96.0,7.0,False,3.0,0.731219,-0.125276,134.0,7.0,False,0.0,A2M-AS1,gene
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000070476,2221.0,5.0,False,0.0,0.981070,0.281466,12827.0,5.0,True,0.0,ZXDC,gene
ENSG00000203995,13.0,0.0,False,0.0,1.400018,0.963488,124.0,0.0,False,0.0,ZYG11A,gene
ENSG00000162378,2290.0,14.0,False,0.0,0.943422,0.220178,11061.0,14.0,True,0.0,ZYG11B,gene


---

---

## 3. Apply the schema requirements:

# <ins>X

Raw and normal count matrices are in the object, no additional adjustment is needed!

# <ins>obs

In [40]:
adata.obs.columns

Index(['batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores',
       'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden',
       'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype',
       'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection',
       'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R',
       'C1_leiden_R_temp', 'project'],
      dtype='object')

In [41]:
adata.obs

,batch,percent_mito,n_counts,n_genes,doublet_scores,bh_pval,transgene,S_score,G2M_score,phase,...,Souporcell3_status,Souporcell3_assignment,stage,dissection,chemistry,old_celltype,C1_leiden,C1_leiden_R,C1_leiden_R_temp,project
AAACCTGAGCGGATCA-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.058898,42167.0,5768,0.060241,0.805722,Ctrl,0.158219,0.756359,G2M,...,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid
AAACCTGAGGAATCGC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.064718,35334.0,5888,0.163418,0.805722,Ctrl,0.181012,0.414132,G2M,...,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid
AAACCTGAGTACGTTC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.077310,17449.0,3611,0.211679,0.805722,Ctrl,-0.057951,-0.129815,G1,...,singlet,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.062381,19660.0,4216,0.096234,0.805722,Ctrl,-0.009915,-0.053740,G1,...,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.063852,33501.0,5109,0.175355,0.805722,Ctrl,-0.080147,-0.081124,G1,...,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGACTTCAACGATCT-5891STDY9030810,5891STDY9030810,0.142772,1435.0,1128,0.082569,0.895486,NaN,0.128544,-0.034317,S,...,nan,nan,9.0,whole lung,Normal,,0,"0,3","0,3",tissue
TTTACTGTCTTACCGC-5891STDY9030810,5891STDY9030810,0.047645,33101.0,6733,0.123684,0.895486,NaN,0.073263,0.077953,G2M,...,nan,nan,11.0,whole lung,Normal,,10,"10,0,1","10,0,1",tissue
TTTGCGCCAAGACACG-5891STDY9030810,5891STDY9030810,0.033311,8677.0,3355,0.108696,0.895486,NaN,-0.122121,-0.069196,G1,...,nan,nan,9.0,whole lung,Normal,,10,"10,1,0","10,1,0",tissue
TTTGCGCTCCGTCAAA-5891STDY9030810,5891STDY9030810,0.178876,964.0,692,0.112207,0.895486,NaN,-0.057342,-0.101391,G1,...,nan,nan,9.0,whole lung,Normal,,5,"5,2","5,2",tissue


## assay_ontology_term_id

Assay used in this study is 10x 5' v1. Publication

10x 3' v2	"EFO:0009899"  
10x 3' v3	"EFO:0009922"  
10x 5' v1	"EFO:0011025"  
Smart-seq	"EFO:0008930"  
Smart-seq2	"EFO:0008931"

> Chromium Single Cell 5’ V(D)J Reagent Kits (V1.0 chemistry) were used for scRNAseq library
construction.

In [42]:
adata.obs['assay_ontology_term_id'] = ['EFO:0011025'] * len(adata.obs)

In [43]:
adata.obs['assay_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    EFO:0011025
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    EFO:0011025
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    EFO:0011025
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    EFO:0011025
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    EFO:0011025
                                          ...     
TTGACTTCAACGATCT-5891STDY9030810       EFO:0011025
TTTACTGTCTTACCGC-5891STDY9030810       EFO:0011025
TTTGCGCCAAGACACG-5891STDY9030810       EFO:0011025
TTTGCGCTCCGTCAAA-5891STDY9030810       EFO:0011025
TTTGGTTGTGCCTTGG-5891STDY9030810       EFO:0011025
Name: assay_ontology_term_id, Length: 70495, dtype: object

## cell_type_ontology_term_id

In [44]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['new_celltype']

In [45]:
len(list(adata.obs['cell_type_ontology_term_id'].unique()))

28

In [46]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['Mid stalk',
 'Mid tip',
 'Late stalk',
 'AT2',
 'Late tip',
 'Mid airway progenitor',
 'Pulmonary NE precursor',
 'GHRL+ NE precursor',
 'Mid basal',
 'Late basal',
 'Pulmonary neuroendocrine',
 'MUC5AC+ ASCL1+ progenitor',
 'Early tip',
 'Late airway progenitor',
 'Early stalk',
 'Early airway progenitor',
 'Club',
 'Squamous',
 'AT1',
 'Secretory 2',
 'Secretory 3',
 'Secretory 1',
 'GHRL+ neuroendocrine',
 'SMG',
 'NEUROD1+ pulmonary neuroendocrine',
 'Proximal basal',
 'Secretory progenitors',
 'SMG basal']

In [47]:
mapping = {'Mid stalk': 'CL:0000082',
 'Mid tip': 'CL:0000082',
 'Late stalk': 'CL:0000082',
 'AT2': 'CL:0002063',
 'Late tip': 'CL:0000082', # not late
 'Mid airway progenitor': 'CL:0002368', # closest = respiratory epithelial?
 'Pulmonary NE precursor': 'CL:0000165', # closest = NE cell?
 'GHRL+ NE precursor': 'CL:0000165', # closest = NE cell?
 'Mid basal': 'CL:0000646',  
 'Late basal': 'CL:0000646',
 'Pulmonary neuroendocrine': 'CL:0000165',
 'MUC5AC+ ASCL1+ progenitor': 'CL:0000082', # only progenitor?
 'Early tip': 'CL:0000082', # not early
 'Late airway progenitor': 'CL:0002368', # closest = respiratory epithelial?
 'Early stalk': 'CL:0000082',
 'Early airway progenitor': 'CL:0002368', # closest = respiratory epithelial?
 'Club': 'CL:0000158',
 'Squamous': 'CL:0000082', # endothelial but maybe epithelial?
 'AT1': 'CL:0002062',
 'Secretory 2': 'CL:1000272', # because it was collected from lung?
 'Secretory 3': 'CL:1000272', # because it was collected from lung?
 'Secretory 1': 'CL:1000272', # because it was collected from lung?
 'GHRL+ neuroendocrine': 'CL:0000165',
 'SMG': 'CL:0000082',
 'NEUROD1+ pulmonary neuroendocrine': 'CL:1000223', # lung
 'Proximal basal': 'CL:0000646',
 'Secretory progenitors': 'CL:1000272', # because it was collected from lung?
 'SMG basal': 'CL:0000646'}

In [48]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].map(mapping)

## development_stage_ontology_term_id

In [49]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['stage']

In [50]:
list(adata.obs['stage'].unique())

[nan, 18.0, 22.0, 15.0, 20.0, 6.86, 5.0, 9.0, 11.0]

In [51]:
mapping = {18.0: 'HsapDv:0000055', 
           22.0: 'HsapDv:0000059', 
           15.0: 'HsapDv:0000052', 
           20.0: 'HsapDv:0000057', 
           6.86: 'HsapDv:0000026', 
           5.0: 'HsapDv:0000023', 
           9.0: 'HsapDv:0000046', 
           11.0: 'HsapDv:0000048',
           np.nan: 'unknown'}

In [52]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].map(mapping)

In [53]:
adata.obs['development_stage_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020           unknown
AAACCTGAGGAATCGC-WSSS_F_LNG10282020           unknown
AAACCTGAGTACGTTC-WSSS_F_LNG10282020           unknown
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020           unknown
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020           unknown
                                            ...      
TTGACTTCAACGATCT-5891STDY9030810       HsapDv:0000046
TTTACTGTCTTACCGC-5891STDY9030810       HsapDv:0000048
TTTGCGCCAAGACACG-5891STDY9030810       HsapDv:0000046
TTTGCGCTCCGTCAAA-5891STDY9030810       HsapDv:0000046
TTTGGTTGTGCCTTGG-5891STDY9030810       HsapDv:0000048
Name: development_stage_ontology_term_id, Length: 70495, dtype: object

In [54]:
adata.obs['development_stage_ontology_term_id'].unique()

array(['unknown', 'HsapDv:0000055', 'HsapDv:0000059', 'HsapDv:0000052',
       'HsapDv:0000057', 'HsapDv:0000026', 'HsapDv:0000023',
       'HsapDv:0000046', 'HsapDv:0000048'], dtype=object)

## disease_ontology_term_id

In [55]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461'] * len(adata.obs)

In [56]:
adata.obs['disease_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    PATO:0000461
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    PATO:0000461
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    PATO:0000461
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    PATO:0000461
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    PATO:0000461
                                           ...     
TTGACTTCAACGATCT-5891STDY9030810       PATO:0000461
TTTACTGTCTTACCGC-5891STDY9030810       PATO:0000461
TTTGCGCCAAGACACG-5891STDY9030810       PATO:0000461
TTTGCGCTCCGTCAAA-5891STDY9030810       PATO:0000461
TTTGGTTGTGCCTTGG-5891STDY9030810       PATO:0000461
Name: disease_ontology_term_id, Length: 70495, dtype: object

In [57]:
adata.obs['disease_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    PATO:0000461
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    PATO:0000461
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    PATO:0000461
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    PATO:0000461
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    PATO:0000461
                                           ...     
TTGACTTCAACGATCT-5891STDY9030810       PATO:0000461
TTTACTGTCTTACCGC-5891STDY9030810       PATO:0000461
TTTGCGCCAAGACACG-5891STDY9030810       PATO:0000461
TTTGCGCTCCGTCAAA-5891STDY9030810       PATO:0000461
TTTGGTTGTGCCTTGG-5891STDY9030810       PATO:0000461
Name: disease_ontology_term_id, Length: 70495, dtype: object

## ethnicity_ontology_term_id

In [58]:
adata.obs['ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [59]:
adata.obs['ethnicity_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    unknown
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    unknown
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    unknown
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    unknown
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    unknown
                                        ...   
TTGACTTCAACGATCT-5891STDY9030810       unknown
TTTACTGTCTTACCGC-5891STDY9030810       unknown
TTTGCGCCAAGACACG-5891STDY9030810       unknown
TTTGCGCTCCGTCAAA-5891STDY9030810       unknown
TTTGGTTGTGCCTTGG-5891STDY9030810       unknown
Name: ethnicity_ontology_term_id, Length: 70495, dtype: object

## is_primary_data

In [60]:
allcells = pd.read_csv('/lustre/scratch117/cellgen/cellgeni/cxgportal_sets/fetal-lung/cellnames_all.csv', index_col=0)

In [61]:
allcells

,batch,dissection,chemistry,percent_mito,n_counts,n_genes,doublet_scores,bh_pval,leiden,phase,...,broad_celltype,assay_ontology_term_id,cell_type_ontology_term_id,development_stage_ontology_term_id,disease_ontology_term_id,ethnicity_ontology_term_id,is_primary_data,organism_ontology_term_id,sex_ontology_term_id,tissue_ontology_term_id
AAACCTGAGACAGGCT-5891STDY8062349,5891STDY8062349,Proximal,Normal,0.042116,8711.0,3101,0.060127,0.826769,0,G1,...,Fibroblast,EFO:0011025,CL:4028004,HsapDv:0000055,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
AAACCTGCACCGAATT-5891STDY8062349,5891STDY8062349,Proximal,Normal,0.042811,8228.0,2992,0.131980,0.826769,0,G1,...,Fibroblast,EFO:0011025,CL:4028004,HsapDv:0000055,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
AAAGATGTCGTGACAT-5891STDY8062349,5891STDY8062349,Proximal,Normal,0.052119,3601.0,1714,0.051245,0.826769,0,G1,...,Fibroblast,EFO:0011025,CL:4028004,HsapDv:0000055,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
AAAGCAACAAACGTGG-5891STDY8062349,5891STDY8062349,Proximal,Normal,0.069157,5707.0,2280,0.083019,0.826769,0,G1,...,Fibroblast,EFO:0011025,CL:4028004,HsapDv:0000055,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
AAAGCAACATACTACG-5891STDY8062349,5891STDY8062349,Proximal,Normal,0.065710,7095.0,2639,0.070568,0.826769,0,G1,...,Fibroblast,EFO:0011025,CL:4028004,HsapDv:0000055,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTCACGGAGCCCTAAT-WSSS_F_LNG8713184,WSSS_F_LNG8713184,Proximal,Normal,0.039884,9581.0,3133,0.330578,0.826283,8,G1,...,Other myeloid,EFO:0011025,CL:0000097,HsapDv:0000052,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
ACGGGCTGTTCAGCGC-WSSS_F_LNG8713185,WSSS_F_LNG8713185,Distal,Normal,0.034237,2706.0,1305,0.394231,0.122739,8,G1,...,Other myeloid,EFO:0011025,CL:0000097,HsapDv:0000052,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
CATCGAACATTAACCG-WSSS_F_LNG8713185,WSSS_F_LNG8713185,Distal,Normal,0.040634,7625.0,2735,0.330578,0.122739,8,G1,...,Other myeloid,EFO:0011025,CL:0000097,HsapDv:0000052,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048
CTCAGAAGTACTCTCC-WSSS_F_LNG8713185,WSSS_F_LNG8713185,Distal,Normal,0.027229,13147.0,3721,0.245161,0.122739,8,S,...,Other myeloid,EFO:0011025,CL:0000097,HsapDv:0000052,PATO:0000461,unknown,True,NCBITaxon:9606,PATO:0000384,UBERON:0002048


In [62]:
adata.obs

,batch,percent_mito,n_counts,n_genes,doublet_scores,bh_pval,transgene,S_score,G2M_score,phase,...,old_celltype,C1_leiden,C1_leiden_R,C1_leiden_R_temp,project,assay_ontology_term_id,cell_type_ontology_term_id,development_stage_ontology_term_id,disease_ontology_term_id,ethnicity_ontology_term_id
AAACCTGAGCGGATCA-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.058898,42167.0,5768,0.060241,0.805722,Ctrl,0.158219,0.756359,G2M,...,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown
AAACCTGAGGAATCGC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.064718,35334.0,5888,0.163418,0.805722,Ctrl,0.181012,0.414132,G2M,...,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown
AAACCTGAGTACGTTC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.077310,17449.0,3611,0.211679,0.805722,Ctrl,-0.057951,-0.129815,G1,...,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.062381,19660.0,4216,0.096234,0.805722,Ctrl,-0.009915,-0.053740,G1,...,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.063852,33501.0,5109,0.175355,0.805722,Ctrl,-0.080147,-0.081124,G1,...,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGACTTCAACGATCT-5891STDY9030810,5891STDY9030810,0.142772,1435.0,1128,0.082569,0.895486,NaN,0.128544,-0.034317,S,...,,0,"0,3","0,3",tissue,EFO:0011025,CL:0000082,HsapDv:0000046,PATO:0000461,unknown
TTTACTGTCTTACCGC-5891STDY9030810,5891STDY9030810,0.047645,33101.0,6733,0.123684,0.895486,NaN,0.073263,0.077953,G2M,...,,10,"10,0,1","10,0,1",tissue,EFO:0011025,CL:0000165,HsapDv:0000048,PATO:0000461,unknown
TTTGCGCCAAGACACG-5891STDY9030810,5891STDY9030810,0.033311,8677.0,3355,0.108696,0.895486,NaN,-0.122121,-0.069196,G1,...,,10,"10,1,0","10,1,0",tissue,EFO:0011025,CL:0000165,HsapDv:0000046,PATO:0000461,unknown
TTTGCGCTCCGTCAAA-5891STDY9030810,5891STDY9030810,0.178876,964.0,692,0.112207,0.895486,NaN,-0.057342,-0.101391,G1,...,,5,"5,2","5,2",tissue,EFO:0011025,CL:0002368,HsapDv:0000046,PATO:0000461,unknown


In [63]:
adata.obs['is_primary_data'] = [False] * len(adata.obs)

In [64]:
len(set(adata.obs_names).intersection(set(allcells.index)))

9096

In [65]:
adata.obs.loc[set(adata.obs_names).difference(set(allcells.index)),'is_primary_data'] = True

In [66]:
adata.obs['is_primary_data']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020     True
AAACCTGAGGAATCGC-WSSS_F_LNG10282020     True
AAACCTGAGTACGTTC-WSSS_F_LNG10282020     True
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020     True
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020     True
                                       ...  
TTGACTTCAACGATCT-5891STDY9030810       False
TTTACTGTCTTACCGC-5891STDY9030810       False
TTTGCGCCAAGACACG-5891STDY9030810       False
TTTGCGCTCCGTCAAA-5891STDY9030810       False
TTTGGTTGTGCCTTGG-5891STDY9030810       False
Name: is_primary_data, Length: 70495, dtype: bool

In [67]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

## organism_ontology_term_id

In [68]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [69]:
adata.obs['organism_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    NCBITaxon:9606
                                            ...      
TTGACTTCAACGATCT-5891STDY9030810       NCBITaxon:9606
TTTACTGTCTTACCGC-5891STDY9030810       NCBITaxon:9606
TTTGCGCCAAGACACG-5891STDY9030810       NCBITaxon:9606
TTTGCGCTCCGTCAAA-5891STDY9030810       NCBITaxon:9606
TTTGGTTGTGCCTTGG-5891STDY9030810       NCBITaxon:9606
Name: organism_ontology_term_id, Length: 70495, dtype: object

In [70]:
adata.obs['organism_ontology_term_id'] = pd.Categorical(adata.obs['organism_ontology_term_id'])

In [71]:
adata.obs['organism_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    NCBITaxon:9606
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    NCBITaxon:9606
                                            ...      
TTGACTTCAACGATCT-5891STDY9030810       NCBITaxon:9606
TTTACTGTCTTACCGC-5891STDY9030810       NCBITaxon:9606
TTTGCGCCAAGACACG-5891STDY9030810       NCBITaxon:9606
TTTGCGCTCCGTCAAA-5891STDY9030810       NCBITaxon:9606
TTTGGTTGTGCCTTGG-5891STDY9030810       NCBITaxon:9606
Name: organism_ontology_term_id, Length: 70495, dtype: category
Categories (1, object): ['NCBITaxon:9606']

In [72]:
pd.set_option('display.max_columns', 500)

In [73]:
adata.obs

,batch,percent_mito,n_counts,n_genes,doublet_scores,bh_pval,transgene,S_score,G2M_score,phase,leiden,predicted_hi,new_celltype,__is_in_cluster__,phenotype,Souporcell3_status,Souporcell3_assignment,stage,dissection,chemistry,old_celltype,C1_leiden,C1_leiden_R,C1_leiden_R_temp,project,assay_ontology_term_id,cell_type_ontology_term_id,development_stage_ontology_term_id,disease_ontology_term_id,ethnicity_ontology_term_id,is_primary_data,organism_ontology_term_id
AAACCTGAGCGGATCA-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.058898,42167.0,5768,0.060241,0.805722,Ctrl,0.158219,0.756359,G2M,2,Early cap C3,Mid stalk,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606
AAACCTGAGGAATCGC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.064718,35334.0,5888,0.163418,0.805722,Ctrl,0.181012,0.414132,G2M,10,Late basal C1,Mid stalk,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606
AAACCTGAGTACGTTC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.077310,17449.0,3611,0.211679,0.805722,Ctrl,-0.057951,-0.129815,G1,9,Late basal C1,Mid tip,False,EGLN3+CA9+ Ctrl,singlet,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.062381,19660.0,4216,0.096234,0.805722,Ctrl,-0.009915,-0.053740,G1,18,Esophagus TRH+ epithelial C1,Mid tip,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.063852,33501.0,5109,0.175355,0.805722,Ctrl,-0.080147,-0.081124,G1,10,DC3 C2,Mid stalk,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGACTTCAACGATCT-5891STDY9030810,5891STDY9030810,0.142772,1435.0,1128,0.082569,0.895486,NaN,0.128544,-0.034317,S,1,NaN,Mid stalk,NaN,nan,nan,nan,9.0,whole lung,Normal,,0,"0,3","0,3",tissue,EFO:0011025,CL:0000082,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606
TTTACTGTCTTACCGC-5891STDY9030810,5891STDY9030810,0.047645,33101.0,6733,0.123684,0.895486,NaN,0.073263,0.077953,G2M,1,NaN,Pulmonary NE precursor,NaN,nan,nan,nan,11.0,whole lung,Normal,,10,"10,0,1","10,0,1",tissue,EFO:0011025,CL:0000165,HsapDv:0000048,PATO:0000461,unknown,False,NCBITaxon:9606
TTTGCGCCAAGACACG-5891STDY9030810,5891STDY9030810,0.033311,8677.0,3355,0.108696,0.895486,NaN,-0.122121,-0.069196,G1,1,NaN,GHRL+ neuroendocrine,NaN,nan,nan,nan,9.0,whole lung,Normal,,10,"10,1,0","10,1,0",tissue,EFO:0011025,CL:0000165,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606
TTTGCGCTCCGTCAAA-5891STDY9030810,5891STDY9030810,0.178876,964.0,692,0.112207,0.895486,NaN,-0.057342,-0.101391,G1,1,NaN,Early airway progenitor,NaN,nan,nan,nan,9.0,whole lung,Normal,,5,"5,2","5,2",tissue,EFO:0011025,CL:0002368,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606


## sex_ontology_term_id

**PATO_0000384 for male  
PATO_0000383 for female**

In [74]:
adata.obs['sex_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [75]:
adata.obs[adata.obs['project'] == 'tissue']

,batch,percent_mito,n_counts,n_genes,doublet_scores,bh_pval,transgene,S_score,G2M_score,phase,leiden,predicted_hi,new_celltype,__is_in_cluster__,phenotype,Souporcell3_status,Souporcell3_assignment,stage,dissection,chemistry,old_celltype,C1_leiden,C1_leiden_R,C1_leiden_R_temp,project,assay_ontology_term_id,cell_type_ontology_term_id,development_stage_ontology_term_id,disease_ontology_term_id,ethnicity_ontology_term_id,is_primary_data,organism_ontology_term_id,sex_ontology_term_id
ACAGCTAAGGGTTTCT-5891STDY8062349,5891STDY8062349,0.076047,33497.0,6552,0.083019,0.999862,NaN,-0.049527,-0.115725,G1,1,NaN,Late tip,NaN,nan,nan,nan,18.0,Proximal,Normal,Tip 1 cell C2,1,1,1,tissue,EFO:0011025,CL:0000082,HsapDv:0000055,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
ACCCACTGTATGAAAC-5891STDY8062349,5891STDY8062349,0.146195,3691.0,1604,0.078611,0.999862,NaN,-0.046104,-0.119784,G1,1,NaN,Club,NaN,nan,nan,nan,18.0,Proximal,Normal,Club cell C2,11,"11,0","11,0",tissue,EFO:0011025,CL:0000158,HsapDv:0000055,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
CTAAGACGTATCGCAT-5891STDY8062349,5891STDY8062349,0.056932,10469.0,3426,0.092742,0.999862,NaN,-0.022936,0.085920,G2M,1,NaN,Club,NaN,nan,nan,nan,18.0,Proximal,Normal,Club cell C2,11,"11,0","11,0",tissue,EFO:0011025,CL:0000158,HsapDv:0000055,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
CTAGCCTTCTGAAAGA-5891STDY8062349,5891STDY8062349,0.059323,8309.0,2608,0.083019,0.999862,NaN,-0.090685,-0.104659,G1,1,NaN,Club,NaN,nan,nan,nan,18.0,Proximal,Normal,Club cell C2,11,"11,0","11,0",tissue,EFO:0011025,CL:0000158,HsapDv:0000055,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
CTGAAGTAGCGATGAC-5891STDY8062349,5891STDY8062349,0.060112,16432.0,3183,0.087719,0.999862,NaN,-0.032836,-0.092125,G1,1,NaN,Club,NaN,nan,nan,nan,18.0,Proximal,Normal,Club cell C2,11,"11,0","11,0",tissue,EFO:0011025,CL:0000158,HsapDv:0000055,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGACTTCAACGATCT-5891STDY9030810,5891STDY9030810,0.142772,1435.0,1128,0.082569,0.895486,NaN,0.128544,-0.034317,S,1,NaN,Mid stalk,NaN,nan,nan,nan,9.0,whole lung,Normal,,0,"0,3","0,3",tissue,EFO:0011025,CL:0000082,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
TTTACTGTCTTACCGC-5891STDY9030810,5891STDY9030810,0.047645,33101.0,6733,0.123684,0.895486,NaN,0.073263,0.077953,G2M,1,NaN,Pulmonary NE precursor,NaN,nan,nan,nan,11.0,whole lung,Normal,,10,"10,0,1","10,0,1",tissue,EFO:0011025,CL:0000165,HsapDv:0000048,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
TTTGCGCCAAGACACG-5891STDY9030810,5891STDY9030810,0.033311,8677.0,3355,0.108696,0.895486,NaN,-0.122121,-0.069196,G1,1,NaN,GHRL+ neuroendocrine,NaN,nan,nan,nan,9.0,whole lung,Normal,,10,"10,1,0","10,1,0",tissue,EFO:0011025,CL:0000165,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606,unknown
TTTGCGCTCCGTCAAA-5891STDY9030810,5891STDY9030810,0.178876,964.0,692,0.112207,0.895486,NaN,-0.057342,-0.101391,G1,1,NaN,Early airway progenitor,NaN,nan,nan,nan,9.0,whole lung,Normal,,5,"5,2","5,2",tissue,EFO:0011025,CL:0002368,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606,unknown


In [76]:
len(set(allcells.index).intersection(set(adata.obs_names[adata.obs['project'] == 'tissue'])))

9096

In [77]:
allcells.loc[list(set(allcells.index).intersection(set(adata.obs_names))), 'sex_ontology_term_id']

CTACGTCAGGGAGTAA-5891STDY9030808      PATO:0000383
CGATCGGTCAAGGTAA-WSSS_F_LNG8713181    PATO:0000384
GATCGATAGCTAGCCC-5891STDY9030809      PATO:0000384
AGTGTCAAGTATCTCG-WSSS_F_LNG8713190    PATO:0000384
TGATTTCTCCTTGCCA-5891STDY9030809      PATO:0000384
                                          ...     
CACATAGCAGTACACT-5891STDY9030808      PATO:0000383
GCATGATCAGGCTCAC-WSSS8012016          PATO:0000383
TCTCTAATCACAGTAC-WSSS_F_LNG8713191    PATO:0000384
CTCTGGTGTCTTGATG-5891STDY9030808      PATO:0000383
CGTAGGCTCATCATTC-5891STDY9030809      PATO:0000384
Name: sex_ontology_term_id, Length: 9096, dtype: object

In [78]:
adata.obs.loc[list(set(adata.obs_names).difference(set(allcells.index))),'sex_ontology_term_id'] = 'unknown'

In [79]:
adata.obs.loc[list(set(allcells.index).intersection(set(adata.obs_names))),'sex_ontology_term_id'] = allcells.loc[list(set(allcells.index).intersection(set(adata.obs_names))), 'sex_ontology_term_id']

In [80]:
adata.obs['sex_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020         unknown
AAACCTGAGGAATCGC-WSSS_F_LNG10282020         unknown
AAACCTGAGTACGTTC-WSSS_F_LNG10282020         unknown
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020         unknown
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020         unknown
                                           ...     
TTGACTTCAACGATCT-5891STDY9030810       PATO:0000383
TTTACTGTCTTACCGC-5891STDY9030810       PATO:0000384
TTTGCGCCAAGACACG-5891STDY9030810       PATO:0000383
TTTGCGCTCCGTCAAA-5891STDY9030810       PATO:0000383
TTTGGTTGTGCCTTGG-5891STDY9030810       PATO:0000384
Name: sex_ontology_term_id, Length: 70495, dtype: object

## tissue_ontology_term_id

Lung: UBERON:0002048

In [81]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0002048 (organoid)'] * len(adata.obs)

In [82]:
adata.obs.loc[list(set(allcells.index).intersection(set(adata.obs_names))),'tissue_ontology_term_id'] = 'UBERON:0002048'

In [83]:
adata.obs['tissue_ontology_term_id']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    UBERON:0002048 (organoid)
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    UBERON:0002048 (organoid)
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    UBERON:0002048 (organoid)
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    UBERON:0002048 (organoid)
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    UBERON:0002048 (organoid)
                                                 ...            
TTGACTTCAACGATCT-5891STDY9030810                  UBERON:0002048
TTTACTGTCTTACCGC-5891STDY9030810                  UBERON:0002048
TTTGCGCCAAGACACG-5891STDY9030810                  UBERON:0002048
TTTGCGCTCCGTCAAA-5891STDY9030810                  UBERON:0002048
TTTGGTTGTGCCTTGG-5891STDY9030810                  UBERON:0002048
Name: tissue_ontology_term_id, Length: 70495, dtype: object

---

# <ins>var and raw.var

In [84]:
adata.var

,n_cells-in vitro,highly_variable_n-in vitro,highly_variable-in vitro,Deep_n-in vitro,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable_n-in vitro,highly_variable-in vivo,Deep_n-in vivo,gene_symbols,feature_biotype
gene_ids-WSSS_F_LNG10282020-in vivo,,,,,,,,,,,,
ENSG00000121410,250.0,13.0,False,0.0,0.978903,0.277938,3826.0,13.0,False,1.0,A1BG,gene
ENSG00000268895,39.0,2.0,False,1.0,0.642069,-0.270407,554.0,2.0,False,0.0,A1BG-AS1,gene
ENSG00000148584,35.0,0.0,False,0.0,1.004072,0.318912,5855.0,0.0,True,0.0,A1CF,gene
ENSG00000175899,359.0,28.0,True,21.0,2.273938,2.386176,244.0,28.0,False,0.0,A2M,gene
ENSG00000245105,96.0,7.0,False,3.0,0.731219,-0.125276,134.0,7.0,False,0.0,A2M-AS1,gene
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000070476,2221.0,5.0,False,0.0,0.981070,0.281466,12827.0,5.0,True,0.0,ZXDC,gene
ENSG00000203995,13.0,0.0,False,0.0,1.400018,0.963488,124.0,0.0,False,0.0,ZYG11A,gene
ENSG00000162378,2290.0,14.0,False,0.0,0.943422,0.220178,11061.0,14.0,True,0.0,ZYG11B,gene


## feature_biotype

Adjusted at the beginning!

In [85]:
adata.var

,n_cells-in vitro,highly_variable_n-in vitro,highly_variable-in vitro,Deep_n-in vitro,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable_n-in vitro,highly_variable-in vivo,Deep_n-in vivo,gene_symbols,feature_biotype
gene_ids-WSSS_F_LNG10282020-in vivo,,,,,,,,,,,,
ENSG00000121410,250.0,13.0,False,0.0,0.978903,0.277938,3826.0,13.0,False,1.0,A1BG,gene
ENSG00000268895,39.0,2.0,False,1.0,0.642069,-0.270407,554.0,2.0,False,0.0,A1BG-AS1,gene
ENSG00000148584,35.0,0.0,False,0.0,1.004072,0.318912,5855.0,0.0,True,0.0,A1CF,gene
ENSG00000175899,359.0,28.0,True,21.0,2.273938,2.386176,244.0,28.0,False,0.0,A2M,gene
ENSG00000245105,96.0,7.0,False,3.0,0.731219,-0.125276,134.0,7.0,False,0.0,A2M-AS1,gene
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000070476,2221.0,5.0,False,0.0,0.981070,0.281466,12827.0,5.0,True,0.0,ZXDC,gene
ENSG00000203995,13.0,0.0,False,0.0,1.400018,0.963488,124.0,0.0,False,0.0,ZYG11A,gene
ENSG00000162378,2290.0,14.0,False,0.0,0.943422,0.220178,11061.0,14.0,True,0.0,ZYG11B,gene


In [86]:
#adata2.var['feature_biotype'] = ['gene'] * len(adata2.var.index)

## feature_id

Adjusted at the beginning!

In [87]:
#adata2.var_names = adata2.var["gene_ids"].copy()
#adata2.var_names.name = "feature_id"
#adata2.raw.var_names = adata2.raw.var["gene_ids"].copy()
#adata2.raw.var_names.name = "feature_id"

## feature_is_filtered

True if feature was filtered out in final matrix (X)  
Problem: This should be annotated in 'var', not 'raw.var'. __How to annotate the gene if it is already filtered out in the final matrix?__

In [88]:
adata.var['feature_is_filtered'] = [False] * len(adata.var)

In [89]:
adata.var['feature_is_filtered']

gene_ids-WSSS_F_LNG10282020-in vivo
ENSG00000121410    False
ENSG00000268895    False
ENSG00000148584    False
ENSG00000175899    False
ENSG00000245105    False
                   ...  
ENSG00000070476    False
ENSG00000203995    False
ENSG00000162378    False
ENSG00000159840    False
ENSG00000074755    False
Name: feature_is_filtered, Length: 24653, dtype: bool

# <ins>obsm

In [95]:
adata.obsm

AxisArrays with keys: X_original_umap, X_pca, X_umap, rep

# <ins>uns

In [96]:
adata.uns['schema_version'] = '2.0.0'

In [97]:
adata.uns['title'] = 'Organoid'

In [98]:
adata.uns['X_normalization'] = 'Scanpy'

In [99]:
adata.uns['default_embedding'] = 'X_umap'

---

---

# 4. Final adjustments

Final adjustments include the removal of the slots that were already re-created for the schema.

In [100]:
adata

AnnData object with n_obs × n_vars = 70495 × 24653
    obs: 'batch', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'transgene', 'S_score', 'G2M_score', 'phase', 'leiden', 'predicted_hi', 'new_celltype', '__is_in_cluster__', 'phenotype', 'Souporcell3_status', 'Souporcell3_assignment', 'stage', 'dissection', 'chemistry', 'old_celltype', 'C1_leiden', 'C1_leiden_R', 'C1_leiden_R_temp', 'project', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id'
    var: 'n_cells-in vitro', 'highly_variable_n-in vitro', 'highly_variable-in vitro', 'Deep_n-in vitro', 'dispersions-in vitro', 'dispersions_norm-in vitro', 'n_cells-in vivo', 'highly_variable_n-in vitro', 'highly_variable-in vivo', 'Deep_n-in vivo', 'gene_symbols', 'feature_biotype', 'feature_is_filtered'
    uns: 'Souporcell3_

In [101]:
adata.obs['transgene']

AAACCTGAGCGGATCA-WSSS_F_LNG10282020    Ctrl
AAACCTGAGGAATCGC-WSSS_F_LNG10282020    Ctrl
AAACCTGAGTACGTTC-WSSS_F_LNG10282020    Ctrl
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020    Ctrl
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020    Ctrl
                                       ... 
TTGACTTCAACGATCT-5891STDY9030810        NaN
TTTACTGTCTTACCGC-5891STDY9030810        NaN
TTTGCGCCAAGACACG-5891STDY9030810        NaN
TTTGCGCTCCGTCAAA-5891STDY9030810        NaN
TTTGGTTGTGCCTTGG-5891STDY9030810        NaN
Name: transgene, Length: 70495, dtype: category
Categories (8, object): ['ASCL1', 'Ctrl', 'NEUROD1', 'NEUROG3', 'PAX9', 'RFX6', 'TFAP2A', 'ΔNp63α']

In [102]:
adata.obs

,batch,percent_mito,n_counts,n_genes,doublet_scores,bh_pval,transgene,S_score,G2M_score,phase,leiden,predicted_hi,new_celltype,__is_in_cluster__,phenotype,Souporcell3_status,Souporcell3_assignment,stage,dissection,chemistry,old_celltype,C1_leiden,C1_leiden_R,C1_leiden_R_temp,project,assay_ontology_term_id,cell_type_ontology_term_id,development_stage_ontology_term_id,disease_ontology_term_id,ethnicity_ontology_term_id,is_primary_data,organism_ontology_term_id,sex_ontology_term_id,tissue_ontology_term_id
AAACCTGAGCGGATCA-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.058898,42167.0,5768,0.060241,0.805722,Ctrl,0.158219,0.756359,G2M,2,Early cap C3,Mid stalk,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606,unknown,UBERON:0002048 (organoid)
AAACCTGAGGAATCGC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.064718,35334.0,5888,0.163418,0.805722,Ctrl,0.181012,0.414132,G2M,10,Late basal C1,Mid stalk,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606,unknown,UBERON:0002048 (organoid)
AAACCTGAGTACGTTC-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.077310,17449.0,3611,0.211679,0.805722,Ctrl,-0.057951,-0.129815,G1,9,Late basal C1,Mid tip,False,EGLN3+CA9+ Ctrl,singlet,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606,unknown,UBERON:0002048 (organoid)
AAACCTGAGTCAAGCG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.062381,19660.0,4216,0.096234,0.805722,Ctrl,-0.009915,-0.053740,G1,18,Esophagus TRH+ epithelial C1,Mid tip,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606,unknown,UBERON:0002048 (organoid)
AAACCTGCAAGAAGAG-WSSS_F_LNG10282020,WSSS_F_LNG10282020,0.063852,33501.0,5109,0.175355,0.805722,Ctrl,-0.080147,-0.081124,G1,10,DC3 C2,Mid stalk,False,Default Ctrl,singlet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organoid,EFO:0011025,CL:0000082,unknown,PATO:0000461,unknown,True,NCBITaxon:9606,unknown,UBERON:0002048 (organoid)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGACTTCAACGATCT-5891STDY9030810,5891STDY9030810,0.142772,1435.0,1128,0.082569,0.895486,NaN,0.128544,-0.034317,S,1,NaN,Mid stalk,NaN,nan,nan,nan,9.0,whole lung,Normal,,0,"0,3","0,3",tissue,EFO:0011025,CL:0000082,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606,PATO:0000383,UBERON:0002048
TTTACTGTCTTACCGC-5891STDY9030810,5891STDY9030810,0.047645,33101.0,6733,0.123684,0.895486,NaN,0.073263,0.077953,G2M,1,NaN,Pulmonary NE precursor,NaN,nan,nan,nan,11.0,whole lung,Normal,,10,"10,0,1","10,0,1",tissue,EFO:0011025,CL:0000165,HsapDv:0000048,PATO:0000461,unknown,False,NCBITaxon:9606,PATO:0000384,UBERON:0002048
TTTGCGCCAAGACACG-5891STDY9030810,5891STDY9030810,0.033311,8677.0,3355,0.108696,0.895486,NaN,-0.122121,-0.069196,G1,1,NaN,GHRL+ neuroendocrine,NaN,nan,nan,nan,9.0,whole lung,Normal,,10,"10,1,0","10,1,0",tissue,EFO:0011025,CL:0000165,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606,PATO:0000383,UBERON:0002048
TTTGCGCTCCGTCAAA-5891STDY9030810,5891STDY9030810,0.178876,964.0,692,0.112207,0.895486,NaN,-0.057342,-0.101391,G1,1,NaN,Early airway progenitor,NaN,nan,nan,nan,9.0,whole lung,Normal,,5,"5,2","5,2",tissue,EFO:0011025,CL:0002368,HsapDv:0000046,PATO:0000461,unknown,False,NCBITaxon:9606,PATO:0000383,UBERON:0002048


In [103]:
adata.obs.dtypes

batch                                 category
percent_mito                           float32
n_counts                               float32
n_genes                                  int64
doublet_scores                         float64
bh_pval                                float64
transgene                             category
S_score                                float32
G2M_score                              float32
phase                                 category
leiden                                category
predicted_hi                          category
new_celltype                          category
__is_in_cluster__                     category
phenotype                             category
Souporcell3_status                    category
Souporcell3_assignment                category
stage                                  float64
dissection                            category
chemistry                             category
old_celltype                          category
C1_leiden    

In [104]:
del adata.obs['bh_pval']
del adata.obs['doublet_scores']
del adata.obs['C1_leiden']
del adata.obs['C1_leiden_R']
del adata.obs['C1_leiden_R_temp']
del adata.obs['__is_in_cluster__']

In [105]:
adata.obs.dtypes

batch                                 category
percent_mito                           float32
n_counts                               float32
n_genes                                  int64
transgene                             category
S_score                                float32
G2M_score                              float32
phase                                 category
leiden                                category
predicted_hi                          category
new_celltype                          category
__is_in_cluster__                     category
phenotype                             category
Souporcell3_status                    category
Souporcell3_assignment                category
stage                                  float64
dissection                            category
chemistry                             category
old_celltype                          category
project                               category
assay_ontology_term_id                  object
cell_type_ont

In [106]:
adata.var.dtypes

n_cells-in vitro               float64
highly_variable_n-in vitro     float64
highly_variable-in vitro      category
Deep_n-in vitro                float64
dispersions-in vitro           float64
dispersions_norm-in vitro      float32
n_cells-in vivo                float64
highly_variable_n-in vitro     float64
highly_variable-in vivo       category
Deep_n-in vivo                 float64
gene_symbols                    object
feature_biotype                 object
feature_is_filtered               bool
dtype: object

In [107]:
# adata.obs
print('obs')
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

# adata.var
print('var')
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

obs
changed stage from float64 to float32
changed n_genes from int64 to int32
var
changed n_cells-in vitro from float64 to float32
changed highly_variable_n-in vitro from float64 to float32
changed Deep_n-in vitro from float64 to float32
changed dispersions-in vitro from float64 to float32
changed n_cells-in vivo from float64 to float32
changed highly_variable_n-in vitro from float64 to float32
changed Deep_n-in vivo from float64 to float32


In [126]:
adata.var = pd.concat([adata.var, adata2.var['highly_variable_n-in vitro']], axis = 1)

In [127]:
adata.var

,n_cells-in vitro,highly_variable-in vitro,Deep_n-in vitro,dispersions-in vitro,dispersions_norm-in vitro,n_cells-in vivo,highly_variable-in vivo,Deep_n-in vivo,gene_symbols,feature_biotype,feature_is_filtered,highly_variable_n-in vitro
gene_ids-WSSS_F_LNG10282020-in vivo,,,,,,,,,,,,
ENSG00000121410,250.0,False,0.0,0.978903,0.277938,3826.0,False,1.0,A1BG,gene,False,13.0
ENSG00000268895,39.0,False,1.0,0.642069,-0.270407,554.0,False,0.0,A1BG-AS1,gene,False,2.0
ENSG00000148584,35.0,False,0.0,1.004072,0.318912,5855.0,True,0.0,A1CF,gene,False,0.0
ENSG00000175899,359.0,True,21.0,2.273938,2.386176,244.0,False,0.0,A2M,gene,False,28.0
ENSG00000245105,96.0,False,3.0,0.731219,-0.125276,134.0,False,0.0,A2M-AS1,gene,False,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000070476,2221.0,False,0.0,0.981070,0.281466,12827.0,True,0.0,ZXDC,gene,False,5.0
ENSG00000203995,13.0,False,0.0,1.400018,0.963488,124.0,False,0.0,ZYG11A,gene,False,0.0
ENSG00000162378,2290.0,False,0.0,0.943422,0.220178,11061.0,True,0.0,ZYG11B,gene,False,14.0


In [7]:
del adata.raw.var['gene_symbols']

In [8]:
del adata.obs['__is_in_cluster__']

In [128]:
adata.write('/lustre/scratch117/cellgen/cellgeni/cxgportal_sets/fetal-lung/final/fl_tiporganoid.h5ad', compression = 'gzip')